In [1]:
# conda deactivate
# source ros

import os
import argparse
from rclpy.serialization import deserialize_message
from rosidl_runtime_py.utilities import get_message
from std_msgs.msg import String
import rosbag2_py
from mcap.reader import make_reader
from mcap_protobuf.decoder import DecoderFactory
from nuscenes.nuscenes import NuScenes

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Traverse all files

In [2]:
mcap_path = '/home/jd/nuscenes2mcap/output'

In [3]:
# Iterate through mcap files and write detections
for root, _,files in os.walk(mcap_path,topdown=True):
    for file in files:

        # Get input filepath
        in_file = os.path.join(root,file)
        print(in_file)

/home/jd/nuscenes2mcap/output/NuScenes-v1.0-mini-scene-0553-megvii.mcap
/home/jd/nuscenes2mcap/output/NuScenes-v1.0-mini-scene-0061-megvii.mcap
/home/jd/nuscenes2mcap/output/NuScenes-v1.0-mini-scene-1077-megvii.mcap
/home/jd/nuscenes2mcap/output/NuScenes-v1.0-mini-scene-0103-megvii.mcap
/home/jd/nuscenes2mcap/output/NuScenes-v1.0-mini-scene-0916-megvii.mcap
/home/jd/nuscenes2mcap/output/NuScenes-v1.0-mini-scene-1094-megvii.mcap
/home/jd/nuscenes2mcap/output/NuScenes-v1.0-mini-scene-0655-megvii.mcap
/home/jd/nuscenes2mcap/output/NuScenes-v1.0-mini-scene-1100-megvii.mcap
/home/jd/nuscenes2mcap/output/NuScenes-v1.0-mini-scene-0757-megvii.mcap
/home/jd/nuscenes2mcap/output/NuScenes-v1.0-mini-scene-0796-megvii.mcap


# Examine a single file

In [6]:
in_file = '/home/jd/nuscenes2mcap/output/NuScenes-v1.0-mini-scene-0061-megvii.mcap'
topics=['/markers/annotations','/markers/detections']
data_dir = '/home/jd/nuScenes/data'
name="v1.0-mini"

nusc = NuScenes(version=name, dataroot=str(data_dir), verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.437 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [25]:
# REORDER INTO DICT INDEXED BY OBJECT, not as mcap indexed by time...

reader = make_reader(open(in_file, 'br'), decoder_factories=[DecoderFactory()])

epoch = 0
t = dict()
objects = dict()

for schema, channel, message, msg in reader.iter_decoded_messages(topics=topics):
        
    if channel.topic =='/markers/annotations':
#         print("Annotation step: " + str(annotation_count))
        
        # Get timestamp for this epoch
        tstamp = msg.entities[0].timestamp
        if tstamp not in t.values():
            t[epoch] = tstamp
            epoch+=1
        
        
        # For each actual annotation in the message


        # Get epoch from timestamp
        current_epoch = list(t.keys())[list(t.values()).index(tstamp)]       
        
        for entity in msg.entities:
            # Get entity data
            obj_id = entity.id
            
            # Get spatial data
            pos = entity.cubes[0].pose.position
            rot = entity.cubes[0].pose.orientation
            size = entity.cubes[0].size
            
            # Get semantic data
            cat = entity.metadata[0].value
            attribute = nusc.get('attribute',entity.metadata[1].value)['name'] if entity.metadata[1].value else ''
            
            # Initialize dict entry for this object if not done already
            if obj_id not in objects.keys():
                objects[obj_id] = dict()
                objects[obj_id]['category'] = cat
                objects[obj_id]['states'] = dict()
          
            objects[obj_id]['states'][current_epoch] = {
                    "pos": pos,
                    "rot": rot,
                    "size": size,
                    "att": attribute
                }
        
        annotation_count +=1


In [24]:
print(objects.keys())
objects['6dd2']

dict_keys(['6dd2', '48d5', 'bd26', '6e04', '4a5a', 'f4b2', 'c08b', '085f', 'bbd6', '03e8', '3c7d', '91f0', '3843', 'f138', '85d7', 'be2c', 'bc38', '8171', 'e91a', '56a7', '03d0', '8705', '16dc', '038a', '2ce1', '1004', 'cfd5', '4c8f', 'd16b', '3b54', '221d', '37f1', '10b5', '1610', '59cf', '2f59', '8524', 'c074', '01c0', '00cf', '4264', '0573', '7c80', 'e3c5', '9f6a', '4080', 'b4db', '58ce', 'cf04', '62bf', 'da62', 'e2f6', 'a600', 'd7e8', '7ba3', 'ac26', '95fc', 'd7f9', '0e79', 'e900', '963c', 'b77d', '3d03', '02e0', '28a7', 'c195', '91b6', '808a', '5ae0', 'c404', '5837', 'd29b', 'b061', 'ba2f', 'b17a', '4005', '0f74', '06b2', '61f9', '86ed', '1bc4', '5952', 'ce99', '7160', '37ae', 'ac81', '8479', '4b77', 'a017', '291e', 'fdd3', '47bc', 'eb3d', '26c3', '1056', '7f0a', '61dd', 'e3b2', 'daf0', '7367', 'c6b0', 'fbbe', 'c0c7', 'a3d0', 'a004', '4e5e', '6f6d', '36f5', '0704', 'e374', '918b', '9909', 'dad6', 'e80c', '32db', 'e59d', '2821', 'be00', 'f8fe', 'd7cd', '8630', 'b1aa', '9753', '2290

{'category': 'human.pedestrian.adult',
 'states': {0: {'pos': x: 373.256
   y: 1130.419
   z: 0.8,
   'rot': z: -0.18301629506281616
   w: 0.9831098797903927,
   'size': x: 0.669
   y: 0.621
   z: 1.642,
   'att': 'pedestrian.standing'},
  1: {'pos': x: 373.256
   y: 1130.419
   z: 0.81,
   'rot': z: -0.18301629506281616
   w: 0.9831098797903927,
   'size': x: 0.669
   y: 0.621
   z: 1.642,
   'att': 'pedestrian.standing'},
  2: {'pos': x: 373.256
   y: 1130.419
   z: 0.82,
   'rot': z: -0.18301629506281616
   w: 0.9831098797903927,
   'size': x: 0.669
   y: 0.621
   z: 1.642,
   'att': 'pedestrian.standing'},
  3: {'pos': x: 373.256
   y: 1130.419
   z: 0.83,
   'rot': z: -0.18301629506281616
   w: 0.9831098797903927,
   'size': x: 0.669
   y: 0.621
   z: 1.642,
   'att': 'pedestrian.standing'},
  4: {'pos': x: 373.256
   y: 1130.419
   z: 0.84,
   'rot': z: -0.18301629506281616
   w: 0.9831098797903927,
   'size': x: 0.669
   y: 0.621
   z: 1.642,
   'att': 'pedestrian.standing'},
  

# Populate the graph with object factors

In [ ]:
# for each object in objects

# iterate through states

# add factors fo